In [1]:
#Load packages
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm

from sklearn import datasets
from sklearn import linear_model
from sklearn.model_selection import train_test_split

import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
%matplotlib inline


C:\Users\emory\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#Load the training data
col_names = ['A', 'B', 'deltaI', 'deltaR', 'E', 'k']
peak_names = ['peak1', 'peak2', 'peak3', 'peak4', 'peak5']
locs_names = ['loc1', 'loc2', 'loc3', 'loc4', 'loc5']
width_names = ['width1', 'width2', 'width3','width4', 'width5']
prom_names = ['prom1', 'prom2', 'prom3', 'prom4', 'prom5']
col_names = col_names + peak_names + locs_names + width_names + prom_names

data = pd.read_csv('LineCutTrainingData052118_fixedfit.csv', header = None, names = col_names)

#The peak information columns have zeros when there weren't 5 peaks

data.head()

FileNotFoundError: File b'LineCutTrainingData052118_fixedfit.csv' does not exist

## Creating the various test, train, data sets

In [ ]:
target = data[['deltaI', 'deltaR']]

train1 = data[['A', 'B', 'E', 'k']]
train2 = data[['A', 'B', 'E']]

##Just the entries with E = -0.4
#train_E_04 = train1[train1['E']==-0.4]
#target_E_04 = target[train1['E']==-0.4]
#train_E_04_a = train_E_04[['A', 'B']] 

#Making a copy of the data frame
df1 = data

columns1 = df1.columns
X1 = df1.drop(columns1[2:4], axis = 1) #everything except target
X2 = X1.drop(columns1[6:], axis = 1) # no additional peak info
X3 = X2.drop(columns1[4:6], axis = 1) # no E, k

Y1 = df1[['deltaR']]
Y2 = df1[['deltaI']]
Y3 = pd.DataFrame.join(Y1,Y2)

#Making another copy of the data frame
df2 = df1

#Initializing a new dataframe to use the average peaks, prominences and widths as features instead of each one
new_df = pd.DataFrame(np.zeros([1000,7]), columns = ['deltaI', 'deltaR', 'avgA', 'avgB', 'avgPeak', 'avgWidth', 'avgProm'])
for i in range(1,1001):
        
        temp_df = df2[9*(i-1):9*i]
        columns = temp_df.columns
        new_df['deltaI'][i-1] = temp_df['deltaI'].mean()
        new_df['deltaR'][i-1] = temp_df['deltaR'].mean()
        
        new_df['avgA'][i-1] = temp_df['A'].mean()
        new_df['avgB'][i-1] = temp_df['B'].mean()
        
        new_df['avgPeak'][i-1] = temp_df[columns[6:11]][temp_df[columns[6:11]]>0].mean().mean()
        
        new_df['avgWidth'][i-1] = temp_df[columns[11:16]][temp_df[columns[11:16]]>0].mean().mean()
        
        new_df['avgProm'][i-1] = temp_df[columns[16:21]][temp_df[columns[16:21]]>0].mean().mean()
        
        
#new_df.head()
new_columns = new_df.columns
X_new_df = new_df.drop(new_columns[0:2], axis=1)
Y_new_df = new_df.drop(new_columns[2:], axis = 1)


#### Splitting each dataset in the previous cell into train and test

In [ ]:
#Random state
rs = 42
#Test size
ts1 = 0.3

X_a_train, X_a_test, Y_a_train, Y_a_test = train_test_split(train1, target, test_size = ts1, random_state = rs)

X_b_train, X_b_test, Y_b_train, Y_b_test = train_test_split(train2, target, test_size = ts1, random_state = rs)

#Splitting X1, Y1 into train and test
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1, Y1, test_size = ts1, random_state = rs)

#Splitting X2, Y2 into train and test
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size = ts1, random_state = rs)

#Splitting X3, Y3 into train and test
X3_train, X3_test, Y3_train, Y3_test = train_test_split(X3, Y3, test_size = ts1, random_state = rs)

#Splitting X_new_df, Y_new_df into train and test
X_new_train, X_new_test, Y_new_train, Y_new_test = train_test_split(X_new_df, Y_new_df, test_size = ts1, random_state = rs)



## Making models

#### Using linear regression from scikit-learn on the minimal data set (only A, phi, E, k as features)

In [ ]:
import pylab
#Initialize and fit model to training data
reg = linear_model.LinearRegression()
reg1 = reg.fit(X_a_train, Y_a_train)

#Make predictions using the testing set
y_pred1 = reg1.predict(X_a_test)

# The coefficients
print('Coefficients: \n', reg1.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y_a_test, y_pred1))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.3f' % r2_score(Y_a_test, y_pred1))

# Plot outputs
plt.scatter(Y_a_test['deltaI'], Y_a_test['deltaR'],  color=['black'], label = 'Testing Data')
plt.plot(y_pred1[:,0], y_pred1[:,1], 'bo', label = 'Predictions')

# add labels to figure
plt.xlabel('deltaI')
plt.ylabel('deltaR')
pylab.legend(loc='upper left')

In [ ]:
iterations = len(Y_a_test['deltaR']) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
SSres_1 = 0 # initializes sum of residuals squared 
for i in range (0,iterations): # for every dot/point
    y_pred1_col = y_pred1[i,1] #get the y value of the line at a specific x value i 
    deltaR = Y_a_test['deltaR'] # get ys of all dots/points 
    deltaR = deltaR.values # take out the indexing, data types, and names
    deltaR = deltaR[i] # get the y value of the dot/point at specfic x value i 
    diff = y_pred1_col - deltaR # find the difference in height 
    diff_squared = (diff)**2 # square the difference 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
    SSres_1 = diff_squared + SSres_1 # add squared residual to sum 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual for ys (deltaRs):', round(residual_average,3)) # display average of residuals 

min1 = min(Y_a_test['deltaR']) # find min y 
max1 = max(Y_a_test['deltaR']) # find max y 
range_Y_a_test = min1 + max1 # calculate range 
print('Range:' , round(range_Y_a_test,3)) # print range 
residual_percent_range_Y_a_test = round(((residual_average/abs(range_Y_a_test))*100),3) # calculate what percent residual is of range
print('Y Residual is this percent of range', residual_percent_range_Y_a_test, '%') # display percent 


SStot_1 = np.var(Y_a_test['deltaR']) # calculate variance/total sum of squares 
SSres_1a = SSres_1/iterations # calculate sum of squares of residuals for ys 
R2_1 = round(1 - (SSres_1a/SStot_1),3) # calculate R squared value for ys 
print('Y (deltaR) variance', R2_1) # print r squared value for ys 
################################################################################
iterations = len(Y_a_test['deltaI']) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
SSres_1x = 0 # initializes sum of residuals squared 
for i in range (0,iterations): # for every dot/point
    y_pred1_row = y_pred1[i,0] #get the y value of the line at a specific x value i 
    deltaI = Y_a_test['deltaI'] # get ys of all dots/points 
    deltaI = deltaI.values # take out the indexing, data types, and names
    deltaI = deltaI[i] # get the y value of the dot/point at specfic x value i 
    diff = y_pred1_row - deltaI # find the difference in height 
    diff_squared = (diff)**2 # square the difference 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
    SSres_1x = diff_squared + SSres_1x # add squared residual to sum 
residual_averagex = residual_sum/iterations # find the average of the residuals 
print(' ')
print('The average residual for ys (deltaRs):', round(residual_averagex,3)) # display average of residuals 

min1x = min(Y_a_test['deltaI']) # find min x
max1x = max(Y_a_test['deltaI']) # find max x 
range_Y_a_testx = min1x + max1x # calculate range 
print('Domain:' , round(range_Y_a_testx,3)) # print range 
residual_percent_range_Y_a_testx = round(((residual_averagex/abs(range_Y_a_testx))*100),3) # calculate what percent residual is of domain
print('X Residual is this percent of range', residual_percent_range_Y_a_testx, '%') # display percent 


SStot_1x = np.var(Y_a_test['deltaI']) # calculate variance/total sum of squares 
SSres_1xa = SSres_1x/iterations # calculate sum of squares of residuals for xs 
R2_1x = round(1 - (SSres_1xa/SStot_1x),3) # calculate R squared value for xs 
print('X (deltaI) variance', R2_1x) # print r squared value for xs 

#### Using linear regression from scikit-learn on the data set with only A, phi and E as features (no k). 

In [ ]:
#Initialize and fit model to training data
reg = linear_model.LinearRegression()
reg2 = reg.fit(X_b_train, Y_b_train)

#Make predictions using the testing set
y_pred2 = reg2.predict(X_b_test)

# The coefficients
print('Coefficients: \n', reg2.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y_b_test, y_pred2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_b_test, y_pred2))

# Plot outputs
plt.scatter(Y_b_test['deltaI'], Y_b_test['deltaR'],  color=['black'])
plt.plot(y_pred2[:,0], y_pred2[:,1], 'bo')

# add labels to figure
plt.xlabel('deltaI')
plt.ylabel('deltaR')
###############################################

i = 0 # resetting i 
iterations = len(Y_b_test['deltaR']) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
SSres_2 = 0
for i in range (0,iterations): # for every dot/point
    y_pred2_col = y_pred2[i,1] #get the y value of the line at a specific x value i 
    deltaR = Y_b_test['deltaR'] # get ys of all dots/points 
    deltaR = deltaR.values # take out the indexing, data types, and names
    deltaR = deltaR[i] # get the y value of the dot/point at specfic x value i 
    diff = y_pred2_col - deltaR # find the difference in height 
    diff_squared = (diff)**2 # square the difference 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
    SSres_2 = diff_squared + SSres_2 # sum the squared residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print(' ')
print('The average residual for ys:', round(residual_average,3)) # display average of residuals 

min2 = min(Y_b_test['deltaR'])
max2 = max(Y_b_test['deltaR'])
range_Y_b_test = min2 + max2
print('Range:' , round(range_Y_b_test,3))
residual_percent_range_Y_b_test = round(((residual_average/abs(range_Y_b_test))*100),3)
print('Y Residual is this percent of range', residual_percent_range_Y_b_test, '%')

SStot_2 = np.var(Y_b_test['deltaR']) # calculate variance/total sum of squares 
SSres_2a = SSres_2/iterations # calculate sum of squares of residuals for ys 
R2_2= round(1 - (SSres_2a/SStot_2),3) # calculate R squared value for ys 
print('Y (deltaR) variance', R2_2) # print r squared value for ys 

################################################################################
iterations = len(Y_b_test['deltaI']) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
SSres_2x = 0 # initializes sum of residuals squared 
for i in range (0,iterations): # for every dot/point
    y_pred1_row = y_pred1[i,0] #get the y value of the line at a specific x value i 
    deltaI = Y_b_test['deltaI'] # get ys of all dots/points 
    deltaI = deltaI.values # take out the indexing, data types, and names
    deltaI = deltaI[i] # get the y value of the dot/point at specfic x value i 
    diff = y_pred1_row - deltaI # find the difference in height 
    diff_squared = (diff)**2 # square the difference 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
    SSres_2x = diff_squared + SSres_2x # add squared residual to sum 
residual_average2x = residual_sum/iterations # find the average of the residuals 
print(' ')
print('The average residual for ys (deltaRs):', round(residual_average2x,3)) # display average of residuals 

min2x = min(Y_b_test['deltaI']) # find min x
max2x = max(Y_b_test['deltaI']) # find max x 
range_Y_b_testx = min2x + max2x # calculate range 
print('Domain:' , round(range_Y_b_testx,3)) # print range 
residual_percent_range_Y_b_testx = round(((residual_average2x/abs(range_Y_b_testx))*100),3) # calculate what percent residual is of domain
print('X Residual is this percent of range', residual_percent_range_Y_b_testx, '%') # display percent 


SStot_2x = np.var(Y_b_test['deltaI']) # calculate variance/total sum of squares 
SSres_2xa = SSres_2x/iterations # calculate sum of squares of residuals for xs 
R2_2x = round(1 - (SSres_2xa/SStot_2x),3) # calculate R squared value for xs 
print('X (deltaI) variance', R2_2x) # print r squared value for xs 

In [ ]:
#Initialize and fit model to training data with new columns
reg = linear_model.LinearRegression()
reg_new_lm = reg.fit(X_new_train, Y_new_train)

#Make predictions using the testing set
y_pred_new_lm = reg_new_lm.predict(X_new_test)

# The coefficients
print('Coefficients: \n', reg_new_lm.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y_new_test, y_pred_new_lm))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_new_test, y_pred_new_lm))

# Plot outputs
plt.scatter(Y_new_test['deltaI'], Y_new_test['deltaR'],  color=['black'], alpha = 0.7)
plt.scatter(y_pred_new_lm[:,0], y_pred_new_lm[:,1], color='blue', linewidth=1, alpha = 0.2)
# add labels to figure
plt.xlabel('deltaI')
plt.ylabel('deltaR')

#############################################################################
i = 0 # resetting i 
iterations = len(Y_new_test['deltaR']) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
SSres_new = 0 # initialize sum of squared residuals 
for i in range (0,iterations): # for every dot/point
    y_pred_new_lm_col = y_pred_new_lm[i,1] #get the y value of the line at a specific x value i 
    deltaR = Y_new_test['deltaR'] # get ys of all dots/points 
    deltaR = deltaR.values # take out the indexing, data types, and names
    deltaR = deltaR[i] # get the y value of the dot/point at specfic x value i 
    diff = y_pred_new_lm_col - deltaR # find the difference in height 
    diff_squared = (diff)**2 # square the difference 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
    SSres_new = diff_squared + SSres_new # sum the squared differences 
residual_average = residual_sum/iterations # find the average of the residuals 
print(' ')
print('The average Y residual:', round(residual_average,3)) # display average of residuals 

min3 = min(Y_new_test['deltaR'])
max3 = max(Y_new_test['deltaR'])
range_Y_new_test = min3 + max3
print('Range:' , round(range_Y_new_test,3))
residual_percent_range_Y_new_test = round(((residual_average/abs(range_Y_new_test))*100),3)
print('Y Residual is this percent of range', residual_percent_range_Y_new_test, '%')

SStot_new = np.var(Y_new_test['deltaR']) # calculate variance/total sum of squares 
SSres_newa = SSres_new/iterations # calculate sum of squares of residuals for ys 
R2_new= round(1 - (SSres_newa/SStot_new),3) # calculate R squared value for ys 
print('Y (deltaR) variance', R2_new) # print r squared value for ys 

################################################################################
iterations = len(Y_new_test['deltaI']) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
SSres_newx = 0 # initializes sum of residuals squared 
for i in range (0,iterations): # for every dot/point
    y_pred_new_lm_row = y_pred_new_lm[i,0] #get the y value of the line at a specific x value i 
    deltaI = Y_new_test['deltaI'] # get ys of all dots/points 
    deltaI = deltaI.values # take out the indexing, data types, and names
    deltaI = deltaI[i] # get the y value of the dot/point at specfic x value i 
    diff = y_pred_new_lm_row - deltaI # find the difference in height 
    diff_squared = (diff)**2 # square the difference 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
    SSres_newx = diff_squared + SSres_newx # add squared residual to sum 
residual_averagenewx = residual_sum/iterations # find the average of the residuals 
print(' ')
print('The average X residual (deltaIs):', round(residual_averagenewx,3)) # display average of residuals 

minnewx = min(Y_new_test['deltaI']) # find min x
maxnewx = max(Y_new_test['deltaI']) # find max x 
range_Y_new_testx = minnewx + maxnewx # calculate range 
print('Domain:' , round(range_Y_new_testx,3)) # print range 
residual_percent_range_Y_new_testx = round(((residual_averagenewx/abs(range_Y_new_testx))*100),3) # calculate what percent residual is of domain
print('X Residual is this percent of range', residual_percent_range_Y_new_testx, '%') # display percent 


SStot_newx = np.var(Y_new_test['deltaI']) # calculate variance/total sum of squares 
SSres_newxa = SSres_newx/iterations # calculate sum of squares of residuals for xs 
R2_newx = round(1 - (SSres_newxa/SStot_newx),3) # calculate R squared value for xs 
print('X (deltaI) variance', R2_newx) # print r squared value for xs 

In [ ]:
X_new_train.columns

# Using statsmodels

In [ ]:
#Initialize and fit OLS model for deltaR 
model = sm.OLS(Y1_train, X1_train)
results = model.fit()

#Make predictions on testing data
predictions = results.predict(X1_test) # make the predictions by the model

results.summary()

In [ ]:
import pylab 
ax = pylab.subplot(121)
ax.plot(X1_test['A'],Y1_test, 'ko', alpha = 0.5)
ax.plot(X1_test['A'], predictions,'bo')
plt.xlabel('A')
plt.ylabel('Delta R')

ax = pylab.subplot(122)
ax.plot(Y1_test,predictions, 'go')
xmin,xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-1,xmax+1)
plt.xlabel('Testing Data_I')
plt.ylabel('Predictions')
plt.margins(0.5,1)
plt.tight_layout()

i = 0 # resetting i 
iterations = len(Y1_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y1_test_ys = Y1_test # get ys of testing data 
    Y1_test_ys = Y1_test_ys.values # take out indexing, data types, names
    Y1_test_y = Y1_test_ys[i] # get y value at specific i 
    predictions_ys = predictions # get ys of all dots/points
    predictions_ys = predictions_ys.values # take out the indexing, data types, and names
    predictions_y = predictions_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y1_test_y - predictions_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY1 = min(Y1_test_ys)
maxY1 = max(Y1_test_ys)
minp = min(predictions)
maxp = max(predictions)
#print('minY1', minY1, 'maxY1', maxY1)
#print('minp', minp, 'maxp', maxp)
rangeY1 = minY1 + maxY1 
rangep = minp-maxp
#print('rangeY1', rangeY1)
#print('rangep', rangep)
residual_percent_rangep = (residual_average/abs(rangep))*100
print('Residual is this percent of range', round(residual_percent_rangep[0],3), '%')
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y1_test, predictions))

In [ ]:
#Initialize and fit OLS model for deltaI 
model_I = sm.OLS(Y2_train, X1_train)
results_I = model_I.fit()

#Make predictions on testing data
predictions_I = results_I.predict(X1_test) # make the predictions by the model

results_I.summary()

In [ ]:
import pylab 
ax = pylab.subplot(121)
ax.plot(X1_test['A'],Y2_test, 'ko', alpha =0.5)
ax.plot(X1_test['A'], predictions_I,'bo', alpha = 0.5)
plt.xlabel('A')
plt.ylabel('Delta I')

ax = pylab.subplot(122)
ax.plot(Y2_test,predictions_I, 'go')
xmin, xmax = plt.xlim()
xmin = -0.2
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_2')
plt.ylabel('Predictions_I')
plt.margins(0.5,1)
plt.tight_layout()
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-1,xmax+0.5)

i = 0 # resetting i 
iterations = len(Y2_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y2_test_ys = Y2_test # get ys of testing data 
    Y2_test_ys = Y2_test_ys.values # take out indexing, data types, names
    Y2_test_y = Y2_test_ys[i] # get y value at specific i 
    predictions_I_ys = predictions_I # get ys of all dots/points
    predictions_I_ys = predictions_I_ys.values # take out the indexing, data types, and names
    predictions_I_y = predictions_I_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y2_test_y - predictions_I_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY2 = min(Y2_test_ys)
maxY2 = max(Y2_test_ys)
minp_I = min(predictions_I)
maxp_I = max(predictions_I)
#print('minY2', minY2, 'maxY2', maxY2)
#print('minp_I', minp_I, 'maxp_I', maxp_I)
rangeY2 = minY2 + maxY2 
rangep_I = minp_I-maxp_I
#print('rangeY2', rangeY2)
#print('rangep_I', rangep_I)
residual_percent_rangep_I = (residual_average/abs(rangep_I))*100
print('Residual is this percent of range', round(residual_percent_rangep_I[0],3), '%')
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y2_test, predictions_I))

In [ ]:
#Initialize and fit model for just deltaR using A, phi, E, k only
model2 = sm.OLS(Y1_train,X2_train)
results2 = model2.fit()

predictions2 = results2.predict(X2_test)
results2.summary()

In [ ]:
import pylab 
ax = pylab.subplot(121)
ax.plot(X2_test['A'],Y1_test, 'ko', alpha = 0.5)
ax.plot(X2_test['A'], predictions2,'bo')
plt.xlabel('A')
plt.ylabel('Delta R')

ax = pylab.subplot(122)
ax.plot(Y1_test,predictions2, 'go')
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_1')
plt.ylabel('Predictions_2')
plt.margins(0.5,1)
plt.tight_layout()
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-1,xmax+1)

i = 0 # resetting i 
iterations = len(Y1_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y1_test_ys = Y1_test # get ys of testing data 
    Y1_test_ys = Y1_test_ys.values # take out indexing, data types, names
    Y1_test_y = Y1_test_ys[i] # get y value at specific i 
    predictions2_ys = predictions2 # get ys of all dots/points
    predictions2_ys = predictions2_ys.values # take out the indexing, data types, and names
    predictions2_y = predictions2_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y1_test_y - predictions2_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY1 = min(Y1_test_ys)
maxY1 = max(Y1_test_ys)
minp2 = min(predictions2)
maxp2 = max(predictions2)
#print('minY1', minY1, 'maxY1', maxY1)
#print('minp2', minp2, 'maxp2', maxp2)
rangeY1 = minY1 + maxY1 
rangep2 = minp2+maxp2
#print('rangeY1', rangeY1)
#print('rangep2', rangep2)
residual_percent_rangep2 = (residual_average/abs(rangep2))*100
print('Residual is this percent of range', round(residual_percent_rangep2[0],3), '%')

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y1_test, predictions2))

In [ ]:
#Initialize and fit model for just deltaI using A, phi, E, k only
model2_I = sm.OLS(Y2_train,X2_train)
results2_I = model2_I.fit()

predictions2_I = results2_I.predict(X2_test)
results2_I.summary()

In [ ]:
import pylab 
ax = pylab.subplot(121)
ax.plot(X2_test['A'],Y2_test, 'ko', alpha =0.5)
ax.plot(X2_test['A'], predictions2_I,'bo')
plt.xlabel('A')
plt.ylabel('Delta I')

ax = pylab.subplot(122)
ax.plot(Y2_test,predictions2_I, 'go')
xmin, xmax = plt.xlim()
xmin = -0.2
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_2')
plt.ylabel('Predictions_2_I')
plt.margins(0.5,1)
plt.tight_layout()
ax.set_xlim(xmin-0.2,xmax+0.2)
ax.set_ylim(xmin-0.5,xmax+0.5)

i = 0 # resetting i 
iterations = len(Y2_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y2_test_ys = Y2_test # get ys of testing data 
    Y2_test_ys = Y2_test_ys.values # take out indexing, data types, names
    Y2_test_y = Y2_test_ys[i] # get y value at specific i 
    predictions2_I_ys = predictions2_I # get ys of all dots/points
    predictions2_I_ys = predictions2_I_ys.values # take out the indexing, data types, and names
    predictions2_I_y = predictions2_I_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y2_test_y - predictions2_I_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY2 = min(Y2_test_ys)
maxY2 = max(Y2_test_ys)
minp2_I = min(predictions2_I)
maxp2_I = max(predictions2_I)
#print('minY2', minY2, 'maxY2', maxY2)
#print('minp2_I', minp2_I, 'maxp2_I', maxp2_I)
rangeY2 = minY2 + maxY2
rangep2_I = minp2_I+maxp2_I
#print('rangeY2', rangeY2)
#print('rangep2_I', rangep2_I)
residual_percent_rangep2_I = (residual_average/abs(rangep2_I))*100
print('Residual is this percent of range', round(residual_percent_rangep2_I[0],3), '%')

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y2_test, predictions2_I))

In [ ]:
#Initialize and fit model for just deltaR using A, phi only
model3 = sm.OLS(Y1_train,X3_train)
results3 = model3.fit()

predictions3 = results3.predict(X3_test)
results3.summary()

In [ ]:
import pylab 
ax = pylab.subplot(121)
ax.plot(X3_test['A'],Y1_test, 'ko', alpha = 0.5)
ax.plot(X3_test['A'], predictions3,'bo')
plt.xlabel('A')
plt.ylabel('Delta R')

ax = pylab.subplot(122)
ax.plot(Y1_test,predictions3, 'go')
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_1')
plt.ylabel('Predictions_3')
plt.margins(0.5,1)
plt.tight_layout()
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.15,xmax+0.7)

i = 0 # resetting i 
iterations = len(Y1_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y1_test_ys = Y1_test # get ys of testing data 
    Y1_test_ys = Y1_test_ys.values # take out indexing, data types, names
    Y1_test_y = Y1_test_ys[i] # get y value at specific i 
    predictions3_ys = predictions3 # get ys of all dots/points
    predictions3_ys = predictions3_ys.values # take out the indexing, data types, and names
    predictions3_y = predictions3_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y1_test_y - predictions3_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY1 = min(Y1_test_ys)
maxY1 = max(Y1_test_ys)
minp3 = min(predictions3)
maxp3 = max(predictions3)
#print('minY1', minY1, 'maxY1', maxY1)
#print('minp3', minp3, 'maxp3', maxp3)
rangeY1 = minY1 + maxY1
rangep3 = minp3-maxp3
#print('rangeY1', rangeY1)
#print('rangep3', rangep3)
residual_percent_rangep3 = (residual_average/abs(rangep3))*100
print('Residual is this percent of range', round(residual_percent_rangep3[0],3), '%')

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y1_test, predictions3))

In [ ]:
#Initialize and fit model for just deltaI using A, phi only
model3_I = sm.OLS(Y2_train,X3_train)
results3_I = model3_I.fit()

predictions3_I = results3_I.predict(X3_test)
results3_I.summary()

In [ ]:
import pylab 
ax = pylab.subplot(121)
ax.plot(X3_test['A'],Y2_test, 'ko', alpha = 0.5)
ax.plot(X3_test['A'], predictions3_I,'bo')
plt.xlabel('A')
plt.ylabel('Delta I')
ax = pylab.subplot(122)
ax.plot(Y2_test,predictions3_I, 'go')
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_2')
plt.ylabel('Predictions_3_I')
plt.margins(0.5,1)
plt.tight_layout()
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.2,xmax+0.1)

i = 0 # resetting i 
iterations = len(Y2_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y2_test_ys = Y2_test # get ys of testing data 
    Y2_test_ys = Y2_test_ys.values # take out indexing, data types, names
    Y2_test_y = Y2_test_ys[i] # get y value at specific i 
    predictions3_I_ys = predictions3_I # get ys of all dots/points
    predictions3_I_ys = predictions3_I_ys.values # take out the indexing, data types, and names
    predictions3_I_y = predictions3_I_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y2_test_y - predictions3_I_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY2 = min(Y2_test_ys)
maxY2 = max(Y2_test_ys)
minp3_I = min(predictions3_I)
maxp3_I = max(predictions3_I)
#print('minY2', minY2, 'maxY2', maxY2)
#print('minp3_I', minp3_I, 'maxp3_I', maxp3_I)
rangeY2 = minY2 + maxY2
rangep3_I = minp3_I-maxp3_I
#print('rangeY2', rangeY2)
#print('rangep3_I', rangep3_I)
residual_percent_rangep3_I = (residual_average/abs(rangep3_I))*100
print('Residual is this percent of range', round(residual_percent_rangep3_I[0],3), '%')

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y2_test, predictions3_I))

In [ ]:
#Initialize and fit model for just deltaI using newer dataframe with average peak info
model_new1_I = sm.OLS(Y_new_train['deltaI'],X_new_train)
results_new1_I = model_new1_I.fit()

predictions_new1_I = results_new1_I.predict(X_new_test)
results_new1_I.summary()

In [ ]:
import pylab 
ax = pylab.subplot(121)
ax.plot(X_new_test['avgA'],Y_new_test['deltaI'], 'ko', alpha = 0.5)
ax.plot(X_new_test['avgA'], predictions_new1_I,'bo')
plt.xlabel('avgA')
plt.ylabel('DeltaI')

ax = pylab.subplot(122)
ax.plot(Y_new_test['deltaI'],predictions_new1_I, 'go')
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_new')
plt.ylabel('Predictions_new_I')
plt.margins(0.5,1)
plt.tight_layout()
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax)

i = 0 # resetting i 
iterations = len(Y_new_test['deltaI']) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y_new_test_ys = Y_new_test['deltaI'] # get ys of testing data 
    Y_new_test_ys = Y_new_test_ys.values # take out indexing, data types, names
    Y_new_test_y = Y_new_test_ys[i] # get y value at specific i 
    predictions_new1_I_ys = predictions_new1_I # get ys of all dots/points
    predictions_new1_I_ys = predictions_new1_I_ys.values # take out the indexing, data types, and names
    predictions_new1_I_y = predictions_new1_I_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y_new_test_y - predictions_new1_I_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', round(residual_average,3)) # display average of residuals 

minY_new = min(Y_new_test_ys)
maxY_new = max(Y_new_test_ys)
minp_new1_I = min(predictions_new1_I)
maxp_new1_I = max(predictions_new1_I)
#print('minY_new', minY_new, 'maxY_new', maxY_new)
#print('minp_new1_I', minp_new1_I, 'maxp_new1_I', maxp_new1_I)
rangeY_new = minY_new + maxY_new
rangep_new1_I = minp_new1_I-maxp_new1_I
#print('rangeY_new', rangeY_new)
#print('rangep_new1_I', rangep_new1_I)
residual_percent_rangep_new1_I = (residual_average/abs(rangep_new1_I))*100
print('Residual is this percent of range', round(residual_percent_rangep_new1_I,3), '%')

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_new_test['deltaI'], predictions_new1_I))

In [ ]:
#Initialize and fit model for just deltaR using newer dataframe with average peak info
model_new1 = sm.OLS(Y_new_train['deltaR'],X_new_train)
results_new1 = model_new1.fit()

predictions_new1 = results_new1.predict(X_new_test)
results_new1.summary2()


In [ ]:

import pylab 
ax = pylab.subplot(121)
ax.plot(X_new_test['avgA'],Y_new_test['deltaR'], 'ko', alpha = 0.5)
ax.plot(X_new_test['avgA'], predictions_new1,'bo')
plt.xlabel('avgA')
plt.ylabel('Delta R')
ax = pylab.subplot(122)
ax.plot(Y_new_test['deltaR'],predictions_new1, 'go')
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_newR')
plt.ylabel('Predictions_new_R')
plt.margins(0.5,1)
plt.tight_layout()
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.5,xmax+0.5)


i = 0 # resetting i 
iterations = len(Y_new_test['deltaR']) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y_new_testR_ys = Y_new_test['deltaR'] # get ys of testing data 
    Y_new_testR_ys = Y_new_testR_ys.values # take out indexing, data types, names
    Y_new_testR_y = Y_new_testR_ys[i] # get y value at specific i 
    predictions_new1_ys = predictions_new1 # get ys of all dots/points
    predictions_new1_ys = predictions_new1_ys.values # take out the indexing, data types, and names
    predictions_new1_y = predictions_new1_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y_new_testR_y - predictions_new1_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', round(residual_average,3)) # display average of residuals 

minY_newR = min(Y_new_testR_ys)
maxY_newR = max(Y_new_testR_ys)
minp_new1 = min(predictions_new1)
maxp_new1 = max(predictions_new1)
#print('minY_newR', minY_newR, 'maxY_newR', maxY_newR)
#print('minp_new1', minp_new1, 'maxp_new1', maxp_new1)
rangeY_newR = minY_newR + maxY_newR
rangep_new1 = minp_new1-maxp_new1
#print('rangeY_newR', rangeY_newR)
#print('rangep_new1', rangep_new1)
residual_percent_rangep_new1 = (residual_average/abs(rangep_new1))*100
print('Residual is this percent of range', round(residual_percent_rangep_new1,3), '%')

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_new_test['deltaI'], predictions_new1_I))

#### Using statsmodels GLM (generalized linear model)

In [ ]:
model4 = sm.GLM(Y1_train,X1_train)
results4 = model4.fit()
predictions4 = results4.predict(X1_test)

results4.summary2()

In [ ]:
import pylab 
ax = pylab.subplot(121)
ax.plot(X1_test['A'],Y1_test, 'ko', alpha = 0.5)
ax.plot(X1_test['A'], predictions4,'bo')
plt.xlabel('A')
plt.ylabel('Delta R')

ax = pylab.subplot(122)
ax.plot(Y1_test,predictions4, 'go')
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_1')
plt.ylabel('Predictions4')
plt.margins(0.5,1)
plt.tight_layout()
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(-2.5,xmax+0.75)

i = 0 # resetting i 
iterations = len(Y1_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y1_test_ys = Y1_test # get ys of testing data 
    Y1_test_ys = Y1_test_ys.values # take out indexing, data types, names
    Y1_test_y = Y1_test_ys[i] # get y value at specific i 
    predictions4_ys = predictions4 # get ys of all dots/points
    predictions4_ys = predictions4_ys.values # take out the indexing, data types, and names
    predictions4_y = predictions4_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y1_test_y - predictions4_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY1 = min(Y1_test_ys)
maxY1 = max(Y1_test_ys)
minp4 = min(predictions4)
maxp4 = max(predictions4)
#print('minY1', minY1, 'maxY1', maxY1)
#print('minp4', minp4, 'maxp4', maxp4)
rangeY1 = minY1 + maxY1
rangep4 = minp4-maxp4
#print('rangeY1', rangeY1)
#print('rangep4', rangep4)
residual_percent_rangep4 = (residual_average/abs(rangep4))*100
print('Residual is this percent of range', round(residual_percent_rangep4[0],3), '%')


# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y1_test, predictions4))

In [ ]:
model4_I = sm.GLM(Y2_train,X1_train)
results4_I = model4_I.fit()
predictions4_I = results4_I.predict(X1_test)

results4_I.summary2()

import pylab 
ax = pylab.subplot(121) # put the delta graph in the first subplot position 
ax.plot(X1_test['A'],Y2_test, 'ko', label = 'Testing') # plot the testing data black 
ax.plot(X1_test['A'], predictions4_I,'bo', alpha = 0.5, label = 'Predictions') # plot the prediction data blue 
plt.xlabel('A')
plt.ylabel('Delta I')
pylab.legend(loc='lower right')

ax = pylab.subplot(122) # put the testing versus predictions in the second subplot position 
ax.plot(Y2_test,predictions4_I, 'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_2')
plt.ylabel('Predictions4_I')
plt.margins(0.5,1)
plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(-2,xmax+0.05)

i = 0 # resetting i 
iterations = len(Y2_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y2_test_ys = Y2_test # get ys of testing data 
    Y2_test_ys = Y2_test_ys.values # take out indexing, data types, names
    Y2_test_y = Y2_test_ys[i]
    predictions4_I_ys = predictions4_I # get ys of all dots/points
    predictions4_I_ys = predictions4_I_ys.values # take out the indexing, data types, and names
    predictions4_I_y = predictions4_I_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y2_test_y - predictions4_I_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY2 = min(Y2_test_ys) # find the minimum y value for testing data 
maxY2 = max(Y2_test_ys) # find the max y value for testing data 
minp4_I = min(predictions4_I) # find the min y value for the prediction data
maxp4_I = max(predictions4_I) # find the max y value for the prediction data 
#print('minY2', minY2, 'maxY2', maxY2)
#print('minp4_I', minp4_I, 'maxp4_I', maxp4_I)
rangeY2 = minY2 + maxY2 # find the range of the testing data
rangep4_I = minp4_I-maxp4_I # find the range of the predictions 
#print('rangeY2', rangeY2)
#print('rangep4_I', rangep4_I)
residual_percent_rangep4_I = (residual_average/abs(rangep4_I))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of range', round(residual_percent_rangep4_I[0],3), '%')


# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y2_test, predictions4_I)) #calculate and display the r squared score 

In [ ]:
model5 = sm.GLM(Y1_train,X2_train)
results5 = model5.fit()
predictions5 = results5.predict(X2_test)

results5.summary()

import pylab 
ax = pylab.subplot(121) # put the delta graph in the first subplot position 
ax.plot(X2_test['A'],Y1_test, 'ko', alpha = 0.5, label = 'Testing') # plot the testing data black 
ax.plot(X2_test['A'], predictions5,'bo', alpha = 0.5, label = 'Predictions') # plot the prediction data blue 
plt.xlabel('A')
plt.ylabel('Delta R')
pylab.legend(loc='lower right')

ax = pylab.subplot(122) # put the testing versus predictions in the second subplot position 
ax.plot(Y1_test,predictions5, 'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_1')
plt.ylabel('Predictions5')
plt.margins(0.5,1)
plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.75)

i = 0 # resetting i 
iterations = len(Y1_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y1_test_ys = Y1_test # get ys of testing data 
    Y1_test_ys = Y1_test_ys.values # take out indexing, data types, names
    Y1_test_y = Y1_test_ys[i] # get y value at specific i 
    predictions5_ys = predictions5 # get ys of all dots/points
    predictions5_ys = predictions5_ys.values # take out the indexing, data types, and names
    predictions5_y = predictions5_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y1_test_y - predictions5_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY1 = min(Y1_test_ys) # find the minimum y value for testing data 
maxY1 = max(Y1_test_ys) # find the max y value for testing data 
minp5 = min(predictions5) # find the min y value for the prediction data
maxp5 = max(predictions5) # find the max y value for the prediction data 
#print('minY1', minY1, 'maxY1', maxY1)
#print('minp5', minp5, 'maxp5', maxp5)
rangeY1 = minY1 + maxY1 # find the range of the testing data
rangep5 = minp5-maxp5 # find the range of the predictions 
#print('rangeY1', rangeY1)
#print('rangep5', rangep5)
residual_percent_rangep5 = (residual_average/abs(rangep5))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of range', round(residual_percent_rangep5[0],3), '%')


# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y1_test, predictions5)) #calculate and display the r squared score 

In [ ]:
model5_I = sm.GLM(Y2_train,X2_train)
results5_I = model5_I.fit()
predictions5_I = results5_I.predict(X2_test)


#plt.scatter(X2_test['A'],Y2_test, color = 'black', alpha = 0.5)
#plt.scatter(X2_test['A'],predictions5_I)

results5_I.summary2()

import pylab 
ax = pylab.subplot(121) # put the delta graph in the first subplot position 
ax.plot(X2_test['A'],Y2_test, 'ko', label = 'Testing') # plot the testing data black 
ax.plot(X2_test['A'], predictions5_I,'bo', alpha = 0.5, label = 'Predictions') # plot the prediction data blue 
plt.xlabel('A')
plt.ylabel('Delta I')
pylab.legend(loc='upper right')

ax = pylab.subplot(122) # put the testing versus predictions in the second subplot position 
ax.plot(Y2_test,predictions5_I, 'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_2')
plt.ylabel('Predictions5_I')
plt.margins(0.5,1)
plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)



i = 0 # resetting i 
iterations = len(Y2_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y2_test_ys = Y2_test # get ys of testing data 
    Y2_test_ys = Y2_test_ys.values # take out indexing, data types, names
    Y2_test_y = Y2_test_ys[i]
    predictions5_I_ys = predictions5_I # get ys of all dots/points
    predictions5_I_ys = predictions5_I_ys.values # take out the indexing, data types, and names
    predictions5_I_y = predictions5_I_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y2_test_y - predictions5_I_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY2 = min(Y2_test_ys) # find the minimum y value for testing data 
maxY2 = max(Y2_test_ys) # find the max y value for testing data 
minp5_I = min(predictions5_I) # find the min y value for the prediction data
maxp5_I = max(predictions5_I) # find the max y value for the prediction data 
#print('minY2', minY2, 'maxY2', maxY2)
#print('minp5_I', minp5_I, 'maxp5_I', maxp5_I)
rangeY2 = minY2 + maxY2 # find the range of the testing data
rangep5_I = minp5_I-maxp5_I # find the range of the predictions 
#print('rangeY2', rangeY2)
#print('rangep5_I', rangep5_I)
residual_percent_rangep5_I = (residual_average/abs(rangep5_I))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of range', round(residual_percent_rangep5_I[0],3), '%')


# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y2_test, predictions5_I)) #calculate and display the r squared score 

In [ ]:
model6 = sm.GLM(Y1_train,X3_train)
results6 = model6.fit()
predictions6 = results6.predict(X3_test)

results6.summary2()

import pylab 
ax = pylab.subplot(121) # put the delta graph in the first subplot position 
ax.plot(X3_test['A'],Y1_test, 'ko', alpha = 0.5, label = 'Testing') # plot the testing data black 
ax.plot(X3_test['A'], predictions6,'bo', alpha = 0.5, label = 'Predictions') # plot the prediction data blue 
plt.xlabel('A')
plt.ylabel('Delta R')
pylab.legend(loc='lower right')

ax = pylab.subplot(122) # put the testing versus predictions in the second subplot position 
ax.plot(Y1_test,predictions6, 'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_1')
plt.ylabel('Predictions6')
plt.margins(0.5,1)
plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.75)


i = 0 # resetting i 
iterations = len(Y1_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y1_test_ys = Y1_test # get ys of testing data 
    Y1_test_ys = Y1_test_ys.values # take out indexing, data types, names
    Y1_test_y = Y1_test_ys[i] # get y value at specific i 
    predictions6_ys = predictions6 # get ys of all dots/points
    predictions6_ys = predictions6_ys.values # take out the indexing, data types, and names
    predictions6_y = predictions6_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y1_test_y - predictions6_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY1 = min(Y1_test_ys) # find the minimum y value for testing data 
maxY1 = max(Y1_test_ys) # find the max y value for testing data 
minp6 = min(predictions6) # find the min y value for the prediction data
maxp6 = max(predictions6) # find the max y value for the prediction data 
#print('minY1', minY1, 'maxY1', maxY1)
#print('minp6', minp6, 'maxp6', maxp6)
rangeY1 = minY1 + maxY1 # find the range of the testing data
rangep6 = minp6-maxp6 # find the range of the predictions 
#print('rangeY1', rangeY1)
#print('rangep6', rangep6)
residual_percent_rangep6 = (residual_average/abs(rangep6))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of range', round(residual_percent_rangep6[0],3), '%')


# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y1_test, predictions6)) #calculate and display the r squared score 

In [ ]:
model6_I = sm.GLM(Y2_train,X3_train)
results6_I = model6_I.fit()
predictions6_I = results6_I.predict(X3_test)

results6_I.summary()


import pylab 
ax = pylab.subplot(121) # put the delta graph in the first subplot position 
ax.plot(X3_test['A'],Y2_test, 'ko', label = 'Testing') # plot the testing data black 
ax.plot(X3_test['A'], predictions6_I,'bo', alpha = 0.5, label = 'Predictions') # plot the prediction data blue 
plt.xlabel('A')
plt.ylabel('Delta I')
pylab.legend(loc='upper right')

ax = pylab.subplot(122) # put the testing versus predictions in the second subplot position 
ax.plot(Y2_test,predictions6_I, 'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_2')
plt.ylabel('Predictions6_I')
plt.margins(0.5,1)
plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)



i = 0 # resetting i 
iterations = len(Y2_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y2_test_ys = Y2_test # get ys of testing data 
    Y2_test_ys = Y2_test_ys.values # take out indexing, data types, names
    Y2_test_y = Y2_test_ys[i]
    predictions6_I_ys = predictions6_I # get ys of all dots/points
    predictions6_I_ys = predictions6_I_ys.values # take out the indexing, data types, and names
    predictions6_I_y = predictions6_I_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y2_test_y - predictions6_I_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', residual_average) # display average of residuals 

minY2 = min(Y2_test_ys) # find the minimum y value for testing data 
maxY2 = max(Y2_test_ys) # find the max y value for testing data 
minp6_I = min(predictions6_I) # find the min y value for the prediction data
maxp6_I = max(predictions6_I) # find the max y value for the prediction data 
#print('minY2', minY2, 'maxY2', maxY2)
#print('minp6_I', minp6_I, 'maxp6_I', maxp6_I)
rangeY2 = minY2 + maxY2 # find the range of the testing data
rangep6_I = minp6_I-maxp6_I # find the range of the predictions 
#print('rangeY2', rangeY2)
#print('rangep6_I', rangep6_I)
residual_percent_rangep6_I = (residual_average/abs(rangep6_I))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of range', round(residual_percent_rangep6_I[0],3), '%')


# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y2_test, predictions6_I)) #calculate and display the r squared score 

In [ ]:
model_new2 = sm.GLM(Y_new_train['deltaR'],X_new_train)
results_new2 = model_new2.fit()
predictions_new2 = results_new2.predict(X_new_test)

results_new2.summary()

import pylab 
ax = pylab.subplot(121) # put the delta graph in the first subplot position 
ax.plot(X_new_test['avgA'],Y_new_test['deltaR'], 'ko', label = 'Testing') # plot the testing data black 
ax.plot(X_new_test['avgA'], predictions_new2,'bo', alpha = 0.5, label = 'Predictions') # plot the prediction data blue 
plt.xlabel('avgA')
plt.ylabel('Delta R')
pylab.legend(loc='lower right')

ax = pylab.subplot(122) # put the testing versus predictions in the second subplot position 
ax.plot(Y_new_test['deltaR'],predictions_new2, 'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_new')
plt.ylabel('Predictions_new2')
plt.margins(0.5,1)
plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)

i = 0 # resetting i 
iterations = len(Y_new_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y_new_test_ys = Y_new_test['deltaR'] # get ys of testing data 
    Y_new_test_ys = Y_new_test_ys.values # take out indexing, data types, names
    Y_new_test_y = Y_new_test_ys[i]
    predictions_new2_ys = predictions_new2 # get ys of all dots/points
    predictions_new2_ys = predictions_new2_ys.values # take out the indexing, data types, and names
    predictions_new2_y = predictions_new2_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y_new_test_y - predictions_new2_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', round(residual_average,3)) # display average of residuals 

minY_new = min(Y_new_test_ys) # find the minimum y value for testing data 
maxY_new = max(Y_new_test_ys) # find the max y value for testing data 
minp_new2 = min(predictions_new2) # find the min y value for the prediction data
maxp_new2 = max(predictions_new2) # find the max y value for the prediction data 
#print('minY_new', minY_new, 'maxY_new', maxY_new)
#print('minp_new2', minp_new2, 'maxp_new2', maxp_new2)
rangeY_new = minY_new + maxY_new # find the range of the testing data
rangep_new2 = minp_new2 -maxp_new2 # find the range of the predictions 
#print('rangeY_new', rangeY_new)
#print('rangep_new2', rangep_new2)
residual_percent_rangep_new2 = (residual_average/abs(rangep_new2))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of range', round(residual_percent_rangep_new2,3), '%')
#Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_new_test['deltaR'], predictions_new2)) #calculate and display the r squared score 

In [ ]:
model_new2_I = sm.GLM(Y_new_train['deltaI'],X_new_train)
results_new2_I = model_new2_I.fit()
predictions_new2_I = results_new2_I.predict(X_new_test)


#plt.scatter(X_new_test['avgA'],Y_new_test['deltaI'], color = 'black', alpha = 0.5)
#plt.scatter(X_new_test['avgA'],predictions_new2_I)

results_new2_I.summary()

import pylab 
ax = pylab.subplot(121) # put the delta graph in the first subplot position 
ax.plot(X_new_test['avgA'],Y_new_test['deltaI'], 'ko', label = 'Testing') # plot the testing data black 
ax.plot(X_new_test['avgA'], predictions_new2_I,'bo', alpha = 0.5, label = 'Predictions') # plot the prediction data blue 
plt.xlabel('avgA')
plt.ylabel('Delta I')
pylab.legend(loc='upper right')

ax = pylab.subplot(122) # put the testing versus predictions in the second subplot position 
ax.plot(Y_new_test['deltaI'],predictions_new2_I, 'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_new')
plt.ylabel('Predictions_new2_I')
plt.margins(0.5,1)
plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)

i = 0 # resetting i 
iterations = len(Y_new_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y_new_test_ys = Y_new_test['deltaI'] # get ys of testing data 
    Y_new_test_ys = Y_new_test_ys.values # take out indexing, data types, names
    Y_new_test_y = Y_new_test_ys[i]
    predictions_new2_I_ys = predictions_new2_I # get ys of all dots/points
    predictions_new2_I_ys = predictions_new2_I_ys.values # take out the indexing, data types, and names
    predictions_new2_I_y = predictions_new2_I_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y_new_test_y - predictions_new2_I_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual:', round(residual_average,3)) # display average of residuals 

minY_new = min(Y_new_test_ys) # find the minimum y value for testing data 
maxY_new = max(Y_new_test_ys) # find the max y value for testing data 
minp_new2_I = min(predictions_new2_I) # find the min y value for the prediction data
maxp_new2_I = max(predictions_new2_I) # find the max y value for the prediction data 
#print('minY_new', minY_new, 'maxY_new', maxY_new)
#print('minp_new2_I', minp_new2_I, 'maxp_new2_I', maxp_new2_I)
rangeY_new = minY_new + maxY_new # find the range of the testing data
rangep_new2_I = minp_new2_I -maxp_new2_I # find the range of the predictions 
#print('rangeY_new', rangeY_new)
#print('rangep_new2_I', rangep_new2_I)
residual_percent_rangep_new2_I = (residual_average/abs(rangep_new2_I))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of range', round(residual_percent_rangep_new2_I,3), '%')
#Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_new_test['deltaI'], predictions_new2_I)) #calculate and display the r squared score 

### Using Decision Trees in scikit-learn to predict deltaI and deltaR simultaneously

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
import pylab

#Initialize the models
regr_1 = DecisionTreeRegressor(max_depth=10, min_samples_leaf=5)
regr_2 = DecisionTreeRegressor(max_depth=20, min_samples_leaf=5)
regr_3 = DecisionTreeRegressor(max_depth=40, min_samples_leaf=5)

#Fit the models
regr_1.fit(X1_train, Y3_train)
regr_2.fit(X1_train, Y3_train)
regr_3.fit(X1_train, Y3_train)


#Print the coefficients or importances
#print('Regression 1 Feature Importance:', regr_1.feature_importances_)
#print('Regression 2 Feature Importance:', regr_2.feature_importances_)
#print('Regression 3 Feature Importance:', regr_3.feature_importances_)

#Use model to predict
y_1 = regr_1.predict(X1_test)
y_2 = regr_2.predict(X1_test)
y_3 = regr_3.predict(X1_test)

#Trying to understand the scoring
#print('Regression 1 Score: ', regr_1.score(X1_test,Y3_test), ', using max depth of 10')
#print('Regression 2 Score: ', regr_2.score(X1_test,Y3_test), ', using max depth of 20')
#print('Regression 3 Score: ', regr_3.score(X1_test,Y3_test),', using max depth of 30')

import pylab 
f,(ax) = plt.subplots(figsize=(15,15))
ax = pylab.subplot(321) # put the delta graph in the first subplot position 
ax.plot(Y3_test.iloc[:, 0], Y3_test.iloc[:, 1], "ko", label="Testing_3") # plot the testing data black 
ax.plot(y_1[:, 0], y_1[:, 1], "bo", label="y_1 (depth10)", alpha = 0.8) # plot the prediction data blue 
plt.xlabel('Delta R')
plt.ylabel('Delta I')
pylab.legend(loc='upper right')
ax.set_ylim(-0.25,1.5)

ax = pylab.subplot(322) # put the testing versus predictions in the second subplot position 
ax.plot(Y3_test, y_1,'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_3')
plt.ylabel('Predictions_y_1 (depth 10)')
plt.margins(0.5,1)
#plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)

ax = pylab.subplot(323) # put the delta graph in the first subplot position 
ax.plot(Y3_test.iloc[:, 0], Y3_test.iloc[:, 1], "ko", label="Testing_3") # plot the testing data black 
ax.plot(y_2[:, 0], y_2[:, 1], "bo", label="y_2 (depth20)", alpha = 0.8) # plot the prediction data blue 
plt.xlabel('Delta R')
plt.ylabel('Delta I')
pylab.legend(loc='upper right')
ax.set_ylim(-0.25,1.5)

ax = pylab.subplot(324) # put the testing versus predictions in the second subplot position 
ax.plot(Y3_test, y_2,'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_3')
plt.ylabel('Predictions_y_2 (depth 20)')
plt.margins(0.5,1)
#plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)
#plt.subplots_adjust(hspace=3)
#plt.subplots_adjust(wspace=1)

ax = pylab.subplot(325) # put the delta graph in the first subplot position 
ax.plot(Y3_test.iloc[:, 0], Y3_test.iloc[:, 1], "ko", label="Testing_3") # plot the testing data black 
ax.plot(y_3[:, 0], y_3[:, 1], "bo", label="y_3 (depth40)", alpha = 0.8) # plot the prediction data blue 
plt.xlabel('Delta R')
plt.ylabel('Delta I')
pylab.legend(loc='upper right')
ax.set_ylim(-0.25,1.5)

ax = pylab.subplot(326) # put the testing versus predictions in the second subplot position 
ax.plot(Y3_test, y_3,'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_3')
plt.ylabel('Predictions_y_3 (depth 40)')
plt.margins(0.5,1)
#plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)

from sklearn.model_selection import cross_val_score

regressor = DecisionTreeRegressor(random_state=0,max_depth=30, min_samples_leaf=5)
cross_val_score(regressor, X1, Y3, cv=10)

i = 0 # resetting i 
iterations = len(Y3_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y3_test_ys = Y3_test['deltaI'] # get ys of testing data 
    Y3_test_ys = Y3_test_ys.values # take out indexing, data types, names
    Y3_test_y = Y3_test_ys[i]
    y1_ys = y_1[:,1] # get ys of all dots/points
    #y1_ys = y1_ys.values # take out the indexing, data types, and names
    y1_y = y1_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y3_test_y - y1_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual for y_1:', round(residual_average,4)) # display average of residuals 

minY_3 = min(Y3_test_ys) # find the minimum y value for testing data 
maxY_3 = max(Y3_test_ys) # find the max y value for testing data 
miny_1 = min(y_1[:,1]) # find the min y value for the prediction data
maxy_1 = max(y_1[:,1]) # find the max y value for the prediction data 
#print('minY_3', minY_3, 'maxY_3', maxY_3)
#print('miny_1', miny_1, 'maxy_1', maxy_1)
rangeY_3 = minY_3 + maxY_3 # find the range of the testing data
rangey_1 = miny_1 -maxy_1 # find the range of the predictions 
#print('rangeY_3', rangeY_3)
#print('rangey_1', rangey_1)
residual_percent_rangey_1 = (residual_average/abs(rangey_1))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of y_1 range', round(residual_percent_rangey_1,3), '%')
#Explained variance score: 1 is perfect prediction
print('Variance score y_1: %.4f' % r2_score(Y3_test, y_1)) #calculate and display the r squared score 
print(' ')
i = 0 # resetting i 
iterations = len(Y3_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y3_test_ys = Y3_test['deltaI'] # get ys of testing data 
    Y3_test_ys = Y3_test_ys.values # take out indexing, data types, names
    Y3_test_y = Y3_test_ys[i]
    y2_ys = y_2[:,1] # get ys of all dots/points
    #y1_ys = y1_ys.values # take out the indexing, data types, and names
    y2_y = y2_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y3_test_y - y2_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual for y_2:', round(residual_average,4)) # display average of residuals 

minY_3 = min(Y3_test_ys) # find the minimum y value for testing data 
maxY_3 = max(Y3_test_ys) # find the max y value for testing data 
miny_2 = min(y_2[:,1]) # find the min y value for the prediction data
maxy_2 = max(y_2[:,1]) # find the max y value for the prediction data 
#print('minY_3', minY_3, 'maxY_3', maxY_3)
#print('miny_2', miny_2, 'maxy_2', maxy_2)
rangeY_3 = minY_3 + maxY_3 # find the range of the testing data
rangey_2 = miny_2 -maxy_2 # find the range of the predictions 
#print('rangeY_3', rangeY_3)
#print('rangey_2', rangey_2)
residual_percent_rangey_2 = (residual_average/abs(rangey_2))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of y_2 range', round(residual_percent_rangey_2,3), '%')
print('Variance score y_2: %.6f' % r2_score(Y3_test, y_2)) #calculate and display the r squared score 
print(' ')
i = 0 # resetting i 
iterations = len(Y3_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y3_test_ys = Y3_test['deltaI'] # get ys of testing data 
    Y3_test_ys = Y3_test_ys.values # take out indexing, data types, names
    Y3_test_y = Y3_test_ys[i]
    y2_ys = y_2[:,1] # get ys of all dots/points
    y3_ys = y_2[:,1] # get ys of all dots/points
    #y1_ys = y1_ys.values # take out the indexing, data types, and names
    y3_y = y3_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y3_test_y - y3_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 
print('The average residual for y_3:', round(residual_average,4)) # display average of residuals 

minY_3 = min(Y3_test_ys) # find the minimum y value for testing data 
maxY_3 = max(Y3_test_ys) # find the max y value for testing data 
miny_3 = min(y_3[:,1]) # find the min y value for the prediction data
maxy_3 = max(y_3[:,1]) # find the max y value for the prediction data 
#print('minY_3', minY_3, 'maxY_3', maxY_3)
#print('miny_3', miny_3, 'maxy_3', maxy_3)
rangeY_3 = minY_3 + maxY_3 # find the range of the testing data
rangey_3 = miny_3 -maxy_3 # find the range of the predictions 
#print('rangeY_3', rangeY_3)
#print('rangey_3', rangey_3)
residual_percent_rangey_3 = (residual_average/abs(rangey_3))*100 # calculate what percent the residual is of the range 
print('Residual is this percent of y_3 range', round(residual_percent_rangey_3,3), '%')
print('Variance score y_3: %.6f' % r2_score(Y3_test, y_3)) #calculate and display the r squared score 

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
import pylab

#Initialize the models
regr_1 = DecisionTreeRegressor(max_depth=10, min_samples_leaf=5)
regr_2 = DecisionTreeRegressor(max_depth=20, min_samples_leaf=5)
regr_3 = DecisionTreeRegressor(max_depth=40, min_samples_leaf=5)

#Fit the models
regr_1.fit(X1_train, Y3_train)
regr_2.fit(X1_train, Y3_train)
regr_3.fit(X1_train, Y3_train)


#Print the coefficients or importances
#print('Regression 1 Feature Importance:', regr_1.feature_importances_)
#print('Regression 2 Feature Importance:', regr_2.feature_importances_)
#print('Regression 3 Feature Importance:', regr_3.feature_importances_)

#Use model to predict
y_1 = regr_1.predict(X1_test)
y_2 = regr_2.predict(X1_test)
y_3 = regr_3.predict(X1_test)

#Trying to understand the scoring
#print('Regression 1 Score: ', regr_1.score(X1_test,Y3_test), ', using max depth of 10')
#print('Regression 2 Score: ', regr_2.score(X1_test,Y3_test), ', using max depth of 20')
#print('Regression 3 Score: ', regr_3.score(X1_test,Y3_test),', using max depth of 30')



from sklearn.model_selection import cross_val_score

regressor = DecisionTreeRegressor(random_state=0,max_depth=30, min_samples_leaf=5)
cross_val_score(regressor, X1, Y3, cv=10)
###################################################  RESIDUALS #######################################################
i = 0 # resetting i 
iterations = len(Y3_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y3_test_ys = Y3_test['deltaI'] # get ys of testing data 
    Y3_test_ys = Y3_test_ys.values # take out indexing, data types, names
    Y3_test_y = Y3_test_ys[i]
    y1_ys = y_1[:,1] # get ys of all dots/points
    #y1_ys = y1_ys.values # take out the indexing, data types, and names
    y1_y = y1_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y3_test_y - y1_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 

minY_3 = min(Y3_test_ys) # find the minimum y value for testing data 
maxY_3 = max(Y3_test_ys) # find the max y value for testing data 
miny_1 = min(y_1[:,1]) # find the min y value for the prediction data
maxy_1 = max(y_1[:,1]) # find the max y value for the prediction data 
#print('minY_3', minY_3, 'maxY_3', maxY_3)
#print('miny_1', miny_1, 'maxy_1', maxy_1)
rangeY_3 = minY_3 + maxY_3 # find the range of the testing data
rangey_1 = miny_1 -maxy_1 # find the range of the predictions 
#print('rangeY_3', rangeY_3)
#print('rangey_1', rangey_1)
residual_percent_rangey_1 = (residual_average/abs(rangey_1))*100 # calculate what percent the residual is of the range 
text1 = 'The average residual for y_1: '+ str(round(residual_average,4)) + '\nResidual is this percent of y_1 range: '+ str(round(residual_percent_rangey_1,3))+ ' %\n' + str('Variance score y_1: %.4f' % r2_score(Y3_test, y_1))
print(' ')
i = 0 # resetting i 
iterations = len(Y3_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y3_test_ys = Y3_test['deltaI'] # get ys of testing data 
    Y3_test_ys = Y3_test_ys.values # take out indexing, data types, names
    Y3_test_y = Y3_test_ys[i]
    y2_ys = y_2[:,1] # get ys of all dots/points
    #y1_ys = y1_ys.values # take out the indexing, data types, and names
    y2_y = y2_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y3_test_y - y2_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals 

minY_3 = min(Y3_test_ys) # find the minimum y value for testing data 
maxY_3 = max(Y3_test_ys) # find the max y value for testing data 
miny_2 = min(y_2[:,1]) # find the min y value for the prediction data
maxy_2 = max(y_2[:,1]) # find the max y value for the prediction data 
#print('minY_3', minY_3, 'maxY_3', maxY_3)
#print('miny_2', miny_2, 'maxy_2', maxy_2)
rangeY_3 = minY_3 + maxY_3 # find the range of the testing data
rangey_2 = miny_2 -maxy_2 # find the range of the predictions 
#print('rangeY_3', rangeY_3)
#print('rangey_2', rangey_2)
residual_percent_rangey_2 = (residual_average/abs(rangey_2))*100 # calculate what percent the residual is of the range 
text2 = 'The average residual for y_2: '+ str(round(residual_average,4)) + '\nResidual is this percent of y_2 range: '+ str(round(residual_percent_rangey_2,3))+ ' %\n' + str('Variance score y_2: %.4f' % r2_score(Y3_test, y_2))
print(' ')
i = 0 # resetting i 
iterations = len(Y3_test) # gets number of dots/points
residual_sum = 0 # initializes sum of residuals
for i in range (0,iterations): # for every dot/point
    Y3_test_ys = Y3_test['deltaI'] # get ys of testing data 
    Y3_test_ys = Y3_test_ys.values # take out indexing, data types, names
    Y3_test_y = Y3_test_ys[i]
    y2_ys = y_2[:,1] # get ys of all dots/points
    y3_ys = y_2[:,1] # get ys of all dots/points
    #y1_ys = y1_ys.values # take out the indexing, data types, and names
    y3_y = y3_ys[i] # get the y value of the dot/point at specfic x value i
    diff = Y3_test_y - y3_y # find the difference in height 
    diff = abs(diff) # take the absolute value 
    residual_sum = diff + residual_sum # add specific residual sum at i to sum of residuals 
residual_average = residual_sum/iterations # find the average of the residuals  

minY_3 = min(Y3_test_ys) # find the minimum y value for testing data 
maxY_3 = max(Y3_test_ys) # find the max y value for testing data 
miny_3 = min(y_3[:,1]) # find the min y value for the prediction data
maxy_3 = max(y_3[:,1]) # find the max y value for the prediction data 
#print('minY_3', minY_3, 'maxY_3', maxY_3)
#print('miny_3', miny_3, 'maxy_3', maxy_3)
rangeY_3 = minY_3 + maxY_3 # find the range of the testing data
rangey_3 = miny_3 -maxy_3 # find the range of the predictions 
#print('rangeY_3', rangeY_3)
#print('rangey_3', rangey_3)
residual_percent_rangey_3 = (residual_average/abs(rangey_3))*100 # calculate what percent the residual is of the range 
text3 = 'The average residual for y_3: '+ str(round(residual_average,4)) + '\nResidual is this percent of y_3 range: '+ str(round(residual_percent_rangey_3,3))+ ' %\n' + str('Variance score y_3: %.4f' % r2_score(Y3_test, y_3))
print(' ')
################################################# PLOTTING ###############################################################
import pylab 
f,(ax) = plt.subplots(figsize=(15,15))
## first two graphs 
ax = pylab.subplot(321) # put the delta graph in the first subplot position 
ax.plot(Y3_test.iloc[:, 0], Y3_test.iloc[:, 1], "ko", label="Testing_3") # plot the testing data black 
ax.plot(y_1[:, 0], y_1[:, 1], "bo", label="y_1 (depth10)", alpha = 0.8) # plot the prediction data blue 
plt.xlabel('Delta R')
plt.ylabel('Delta I')
pylab.legend(loc='upper right')
ax.set_ylim(-0.25,1.5)

ax = pylab.subplot(322) # put the testing versus predictions in the second subplot position 
ax.plot(Y3_test, y_1,'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_3')
plt.ylabel('Predictions_y_1 (depth 10)')
plt.margins(0.5,1)
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)
ax.text(1,0.1, text1, size=12, ha="left", 
         transform=ax.transAxes)

## 2nd row of graphs
ax = pylab.subplot(323) # put the delta graph in the first subplot position 
ax.plot(Y3_test.iloc[:, 0], Y3_test.iloc[:, 1], "ko", label="Testing_3") # plot the testing data black 
ax.plot(y_2[:, 0], y_2[:, 1], "bo", label="y_2 (depth20)", alpha = 0.8) # plot the prediction data blue 
plt.xlabel('Delta R')
plt.ylabel('Delta I')
pylab.legend(loc='upper right')
ax.set_ylim(-0.25,1.5)
 
ax = pylab.subplot(324) # put the testing versus predictions in the second subplot position 
ax.plot(Y3_test, y_2,'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_3')
plt.ylabel('Predictions_y_2 (depth 20)')
plt.margins(0.5,1)
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)
#plt.subplots_adjust(hspace=3)
#plt.subplots_adjust(wspace=1)
ax.text(1,0.1, text2, size=12, ha="left", 
         transform=ax.transAxes)
## 3rd row of graphs 
ax = pylab.subplot(325) # put the delta graph in the first subplot position 
ax.plot(Y3_test.iloc[:, 0], Y3_test.iloc[:, 1], "ko", label="Testing_3") # plot the testing data black 
ax.plot(y_3[:, 0], y_3[:, 1], "bo", label="y_3 (depth40)", alpha = 0.8) # plot the prediction data blue 
plt.xlabel('Delta R')
plt.ylabel('Delta I')
pylab.legend(loc='upper right')
ax.set_ylim(-0.25,1.5)

ax = pylab.subplot(326) # put the testing versus predictions in the second subplot position 
ax.plot(Y3_test, y_3,'go') # plot the testing versus predictions in green 
xmin, xmax = plt.xlim()
plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one
plt.xlabel('Testing Data_3')
plt.ylabel('Predictions_y_3 (depth 40)')
plt.margins(0.5,1)
#plt.tight_layout() # does not allow axis labels to overlap
ax.set_xlim(xmin-0.5,xmax+0.5)
ax.set_ylim(xmin-0.25,xmax+0.05)
ax.text(1,0.1, text3, size=12, ha="left", 
         transform=ax.transAxes)

In [ ]:
#Decision tree with different X features - average peak info
#Initialize the models
regr_1a = DecisionTreeRegressor(max_depth=10, min_samples_leaf=5)
regr_2a = DecisionTreeRegressor(max_depth=20, min_samples_leaf=5)
regr_3a = DecisionTreeRegressor(max_depth=30, min_samples_leaf=5)

#Fit the models
regr_1a.fit(X_new_train, Y_new_train)
regr_2a.fit(X_new_train, Y_new_train)
regr_3a.fit(X_new_train, Y_new_train)


#Print the coefficients or importances
print('Regression 1a Feature Importance:', regr_1a.feature_importances_)
print('Regression 2a Feature Importance:', regr_2a.feature_importances_)
print('Regression 3a Feature Importance:', regr_3a.feature_importances_)

#Use model to predict
y_1a = regr_1a.predict(X_new_test)
y_2a = regr_2a.predict(X_new_test)
y_3a = regr_3a.predict(X_new_test)

#Plot results
s = 25
plt.scatter(Y_new_test.iloc[1:100, 0], Y_new_test.iloc[1:100, 1], c="navy", s=s,edgecolor="black", label="data")

#plt.show()
plt.scatter(y_1a[1:100, 0], y_1a[1:100, 1], c="cornflowerblue", s=s,edgecolor="red", label="data", alpha = 0.8)
plt.scatter(y_2a[1:100, 0], y_2a[1:100, 1], c="red", s=10,edgecolor="red", label="data")
plt.scatter(y_3a[1:100, 0], y_3a[1:100, 1], c="orange", s=10,edgecolor="black", label="data")

#Trying to understand the scoring
print('Regression 1 Score: ', regr_1a.score(X_new_test,Y_new_test), ', using max depth of 10')
print('Regression 2 Score: ', regr_2a.score(X_new_test,Y_new_test), ', using max depth of 20')
print('Regression 3 Score: ', regr_3a.score(X_new_test,Y_new_test),', using max depth of 30')

regressora = DecisionTreeRegressor(random_state=0,max_depth=30, min_samples_leaf=5)
print('Cross Validation Scores: ', cross_val_score(regressora, X_new_df, Y_new_df, cv=10))

In [ ]:
#Decision tree with different X features - no peak info (X2)
#Initialize the models
regr_1b = DecisionTreeRegressor(max_depth=10, min_samples_leaf=5)
regr_2b = DecisionTreeRegressor(max_depth=20, min_samples_leaf=5)
regr_3b = DecisionTreeRegressor(max_depth=30, min_samples_leaf=5)

#Fit the models
regr_1b.fit(X2_train, Y3_train)
regr_2b.fit(X2_train, Y3_train)
regr_3b.fit(X2_train, Y3_train)


#Print the coefficients or importances
print('Regression 1b Feature Importance:', regr_1b.feature_importances_)
print('Regression 2b Feature Importance:', regr_2b.feature_importances_)
print('Regression 3b Feature Importance:', regr_3b.feature_importances_)

#Use model to predict
y_1b = regr_1b.predict(X2_test)
y_2b = regr_2b.predict(X2_test)
y_3b = regr_3b.predict(X2_test)

#Plot results
s = 25
plt.scatter(Y3_test.iloc[1:100, 0], Y3_test.iloc[1:100, 1], c="navy", s=s,edgecolor="black", label="data")

#plt.show()
plt.scatter(y_1b[1:100, 0], y_1b[1:100, 1], c="cornflowerblue", s=s,edgecolor="red", label="data", alpha = 0.8)
plt.scatter(y_2b[1:100, 0], y_2b[1:100, 1], c="red", s=10,edgecolor="red", label="data")
plt.scatter(y_3b[1:100, 0], y_3b[1:100, 1], c="orange", s=10,edgecolor="black", label="data")

#Trying to understand the scoring
print('Regression 1 Score: ', regr_1b.score(X2_test,Y3_test), ', using max depth of 10')
print('Regression 2 Score: ', regr_2b.score(X2_test,Y3_test), ', using max depth of 20')
print('Regression 3 Score: ', regr_3b.score(X2_test,Y3_test),', using max depth of 30')

regressorb = DecisionTreeRegressor(random_state=0,max_depth=30, min_samples_leaf=5)
print('Cross Validation Scores: ', cross_val_score(regressorb, X2, Y3, cv=10))

In [ ]:
#Decision tree with different X features - just A, phi (X3)
#Initialize the models
regr_1c = DecisionTreeRegressor(max_depth=10, min_samples_leaf=5)
regr_2c = DecisionTreeRegressor(max_depth=20, min_samples_leaf=5)
regr_3c = DecisionTreeRegressor(max_depth=30, min_samples_leaf=5)

#Fit the models
regr_1c.fit(X3_train, Y3_train)
regr_2c.fit(X3_train, Y3_train)
regr_3c.fit(X3_train, Y3_train)


#Print the coefficients or importances
print('Regression 1c Feature Importance:', regr_1c.feature_importances_)
print('Regression 2c Feature Importance:', regr_2c.feature_importances_)
print('Regression 3c Feature Importance:', regr_3c.feature_importances_)

#Use model to predict
y_1c = regr_1c.predict(X3_test)
y_2c = regr_2c.predict(X3_test)
y_3c = regr_3c.predict(X3_test)

#Plot results
s = 25
plt.scatter(Y3_test.iloc[1:100, 0], Y3_test.iloc[1:100, 1], c="navy", s=s,edgecolor="black", label="data")

#plt.show()
plt.scatter(y_1c[1:100, 0], y_1c[1:100, 1], c="cornflowerblue", s=s,edgecolor="red", label="data", alpha = 0.8)
plt.scatter(y_2c[1:100, 0], y_2c[1:100, 1], c="red", s=10,edgecolor="red", label="data")
plt.scatter(y_3c[1:100, 0], y_3c[1:100, 1], c="orange", s=10,edgecolor="black", label="data")

#Trying to understand the scoring
print('Regression 1c Score: ', regr_1c.score(X3_test,Y3_test), ', using max depth of 10')
print('Regression 2c Score: ', regr_2c.score(X3_test,Y3_test), ', using max depth of 20')
print('Regression 3c Score: ', regr_3c.score(X3_test,Y3_test),', using max depth of 30')

regressorc = DecisionTreeRegressor(random_state=0,max_depth=30, min_samples_leaf=5)
print('Cross Validation Scores 3c: ', cross_val_score(regressorc, X3, Y3, cv=10))

In [ ]:
#Visualize a Decision Tree
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()

export_graphviz(regr_1c, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names=X3_test.columns)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  

graph.write_png(path = 'treeTest1.png')
Image(graph.create_png())

# Random Forest

One way to improve the effectiveness of decision trees and reduce the risk of overfitting, is to implement a random forest, where there are several decision trees and certain nodes within are randomly removed to avoid having one node affect the results too much. Could also use bagging to avoid overfitting. 


From the scikit-learn page for RandomForestRegressor:
A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and use averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement if bootstrap=True (default).

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rs = 42
regr_rf = RandomForestRegressor(max_depth = 6,  random_state=rs)
regr_rf.fit(X1_train, Y3_train)

print('Score: ', regr_rf.score(X1_test, Y3_test))

In [ ]:
pd.Series(regr_rf.feature_importances_, index= X1_train.columns)

In [ ]:
#Trying with a different dataset - average peak info

regr_rf1 = RandomForestRegressor(max_depth = 10, random_state = rs)
regr_rf1 = regr_rf1.fit(X_new_train, Y_new_train)

print('Score: ', regr_rf1.score(X_new_test, Y_new_test)) #R^2 score 

predict1 = pd.DataFrame(regr_rf1.predict(X_new_test), columns = ['deltaI', 'deltaR'])

plt.scatter(Y_new_test['deltaI'], predict1['deltaI'], alpha=0.2)
xmin, xmax = plt.xlim()
xmin = -0.25


plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one

In [ ]:
plt.scatter(Y_new_test['deltaR'], predict1['deltaR'], alpha=0.2)
xmin, xmax = plt.xlim()
xmin = -2


plt.plot([xmin,xmax], [xmin,xmax], c='r', linewidth = 2)  #line with slope of one


In [ ]:
pd.Series(regr_rf1.feature_importances_, index= X_new_train.columns)


In [ ]:
#Save images of each decision tree in the forest
import six
#from sklearn import tree

dotfile = six.StringIO()
i_tree = 0
for tree_in_forest in regr_rf1.estimators_:
    if (i_tree <= len(regr_rf1.estimators_)):        
        export_graphviz(tree_in_forest, out_file=dotfile, feature_names=X_new_train.columns)
        (pydotplus.graph_from_dot_data(dotfile.getvalue())).write_png('dtree_depth6_'+ str(i_tree) +'.png')
        dotfile = six.StringIO()
        i_tree = i_tree + 1

In [ ]:
#Big Change

In [ ]:
#Ken's test change 

In [ ]:
#Laura made a change too
